In [67]:
import pandas as pd
import sklearn as sk
import numpy as np
from itertools import groupby
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
import warnings
warnings.filterwarnings('ignore')

## Read Wongnai sentiment data
#### Refernce: https://www.kaggle.com/c/wongnai-challenge-review-rating-prediction/data

In [79]:
data_df = pd.read_csv('../my_shared_data_folder/w_review_train.csv',names=['text','score'],sep=';')
print('total review: ',data_df.text.size)
print(data_df['text'][0])
data_df.head()

total review:  40000
ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย 
ลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่
ทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล 
พะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว
เลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) 

คงไม่ไปซ้ำ แพงเกิน 


,text,score
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5


## Encoded raw sentence to array of index (of words) 

### 1. Tokenize sentnce with PythaiNLP's deepcut

In [70]:
t0 = word_tokenize(data_df['text'][0],engine='deepcut')
t0

['ร้าน',
 'อาหาร',
 'ใหญ่',
 'มาก',
 'กกกกกก',
 ' \n',
 'เลี้ยว',
 'เข้า',
 'มา',
 'เจอ',
 'ห้องน้ำ',
 'ก่อน',
 'เลย',
 ' ',
 'เออแปลก',
 'ดี',
 ' \n',
 'ห้องทาน',
 'หลัก',
 'ๆ',
 'อยู่',
 'ชั้น',
 ' ',
 '2',
 ' ',
 'มี',
 'กาแฟ',
 ' ',
 'น้ำผึ้ง',
 ' ',
 'ซึ่ง',
 'ก็',
 'แค่',
 'เอา',
 'น้ำผึ้ง',
 'มา',
 'ราด',
 ' แพงเวอร์',
 ' ',
 'อย่า',
 'สั่ง',
 'เลย',
 ' \n',
 'ลาบ',
 'ไข่',
 'ต้ม',
 ' ',
 'ไข่',
 'มัน',
 'คาว',
 'อะ',
 ' ',
 'เลย',
 'ไม่',
 'ประทับใจ',
 'เท่า',
 'ไหร่\n',
 'ทอด',
 'มัน',
 'หัว',
 'ปลีก',
 'รอบ',
 'อร่อย',
 'ต้อง',
 'เบิ้ล',
 ' \n',
 'พะแนง',
 'ห่อ',
 'ไข่',
 'อร่อย',
 'ดี',
 ' ',
 'เห้ยแต่',
 'ราคา',
 ' ',
 '150',
 'บาท',
 'มัน',
 'เกิน',
 'ไป',
 'นะ',
 ' ',
 'รับ',
 'ไม่',
 'ไหวว\n',
 'เลิก',
 'กิน',
 'แล้ว',
 'มี',
 'ขนม',
 'หวาน',
 'ให้',
 'กิน',
 'ฟรี',
 'เล็กน้อย',
 ' ',
 ')',
 'ขนม',
 'ไทย',
 ')',
 ' \n\n',
 'คง',
 'ไม่',
 'ไป',
 'ซ้ำ',
 ' ',
 'แพงเกิน',
 ' ']

### 2. get stop word, exception word

In [71]:
stop_word = thai_stopwords()
list(stop_word)[0:10]

['ถึงเมื่อไร',
 'บน',
 'ก็แล้วแต่',
 'ระยะๆ',
 'เพียงเพื่อ',
 'ให้แก่',
 'ครั้งคราว',
 'ไม่',
 'ตลอดเวลา',
 'ใช่']

In [6]:
t0_cleaned = []
escape_word = ['\n','',' ',')','(']
for word in t0:
    word = word.strip().replace('\n','')
    if word not in stop_word and word not in escape_word:
        t0_cleaned.append(word)

In [65]:
print('original:', len(t0))
print('stop word removed:', len(t0_cleaned))

original: 108
stop word removed: 53


In [72]:
## sumarize the function
def tokenize_word(text):
    words_ori = word_tokenize(text,engine='deepcut')
    words_cleaned = []
    escape_word = ['\n','',' ',')','(']
    stop_word = thai_stopwords()
    for word in words_ori:
        word = word.strip().replace('\n','')
        if word not in stop_word and word not in escape_word:
            words_cleaned.append(word)
    return words_cleaned

### 3. Apply tokenization to dataset

#### For the sake of limited resource, only 50 data is using for now

In [73]:
data_df_t = data_df[0:50]
data_df_t['token'] = data_df_t['text'].apply(tokenize_word)

In [74]:
data_df_t.head()

,text,score,token
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3,"[ร้าน, อาหาร, กกกกกก, เลี้ยว, เจอ, ห้องน้ำ, เอ..."
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4,"[อาหาร, อาหาร, จีน, แคะ, หากิน, บ้าน, ตัว, ร้า..."
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3,"[ปอเปี๊ยะ, สด, รู้สึก, หา, กิน, ร้าน, ขาย, ปอเ..."
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5,"[รัานคัพ, เค้ก, เมืองไทย, คน, สงสัย, คัพเค้ก, ..."
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5,"[อร่อย, !, !, !, เดิน, Digital, gateway, ยัก, ..."


### 4. Make vocabulary list
#### 1) Find how many vocabularies are used in this dataset
#### 2) Filter only word exist more than 1
#### 3) Define vocab_list and 'NA' symbol

In [81]:
vocab_w = [item for sublist in data_df_t.token for item in sublist]
vocab_df = pd.DataFrame({'word':vocab_w})
vocab_n = vocab_df['word'].value_counts()
vocab = vocab_n[vocab_n > 1]
vocab_list = ['_'] + list(vocab.keys())
vocab_size = len(vocab_list)
print('vocab size:',vocab_size)
print('vocab list:',vocab_list[0:10])

vocab size: 643
vocab list: ['_', 'ร้าน', 'กิน', 'อาหาร', 'คน', 'ดี', '"', 'เนื้อ', 'ทาน', 'อร่อย']


### 5. Encode words to 'array of word index'

In [82]:
# [vocab_list.index(word) if word in vocab_list else 0 for word in train_data.token[0]]
vocab_onehot = []
for word in data_df_t.token[0]:
    if word in vocab_list:
        vocab_onehot.append(vocab_list.index(word))
    else:
        vocab_onehot.append(0)

In [83]:
def one_hot_word(words):
    return [vocab_list.index(word) if word in vocab_list else 0 for word in words]

In [84]:
data_df_t['token_onehot'] = data_df_t['token'].apply(one_hot_word)
data_df_t.head()

,text,score,token,token_onehot
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3,"[ร้าน, อาหาร, กกกกกก, เลี้ยว, เจอ, ห้องน้ำ, เอ...","[1, 3, 413, 274, 108, 322, 0, 5, 0, 177, 176, ..."
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4,"[อาหาร, อาหาร, จีน, แคะ, หากิน, บ้าน, ตัว, ร้า...","[3, 3, 332, 0, 0, 90, 53, 1, 0, 0, 0, 0, 274, ..."
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3,"[ปอเปี๊ยะ, สด, รู้สึก, หา, กิน, ร้าน, ขาย, ปอเ...","[385, 73, 95, 54, 2, 1, 75, 385, 73, 108, 75, ..."
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5,"[รัานคัพ, เค้ก, เมืองไทย, คน, สงสัย, คัพเค้ก, ...","[0, 36, 544, 4, 362, 189, 36, 96, 189, 458, 36..."
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5,"[อร่อย, !, !, !, เดิน, Digital, gateway, ยัก, ...","[9, 52, 52, 52, 35, 488, 538, 0, 91, 1, 0, 0, ..."


### 6. Let's play with deep-learning
#### We will using only Word embedding layer for detection


In [85]:
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [94]:
max_length = data_df_t.token_onehot.map(len).max()
num_classes = 5
embedding_dimension = 8
X_nopadded = data_df_t.token_onehot
#Padding sequence, thus, every sentence is in the same length
X = pad_sequences(X_nopadded,max_length)
Y = to_categorical(list(data_df_t.score - 1),num_classes=num_classes)
print('longest sentence: ',max_length)
print('Sentence shape: ', X.shape)
print('Result   shape: ',Y.shape)

longest sentence:  310
Sentence shape:  (50, 310)
Result   shape:  (50, 5)


#### Since we don't have a huge dataset. let's split train/test set for 70/30

In [92]:
X_train,X_test,Y_train,Y_test = sk.model_selection.train_test_split(X,Y,train_size=0.7,random_state=1)

In [93]:
X_train

array([[  0,   0,   0, ..., 102,  30,   0],
       [  0,   0,   0, ...,   0,  14,  17],
       [  0,   0,   0, ...,  69, 612,   0],
       ...,
       [  0,   0,   0, ...,   2,  10,  14],
       [  0,   0,   0, ..., 224,  20,   0],
       [  0,   0,   0, ..., 463,   2, 258]], dtype=int32)

#### Define model

In [96]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dimension, input_length=max_length))
model.add(Flatten())
model.add(Dense(num_classes, activation='sigmoid'))

In [97]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 310, 8)            5144      
_________________________________________________________________
flatten_5 (Flatten)          (None, 2480)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 5)                 12405     
Total params: 17,549
Trainable params: 17,549
Non-trainable params: 0
_________________________________________________________________
None


#### Fit the model to train dataset

In [98]:
model.fit(X_train, Y_train, epochs=50, verbose=1)

Epoch 1/50
35/35 [==============================] - 0s 9ms/step - loss: 0.6971 - acc: 0.4629
Epoch 2/50
35/35 [==============================] - 0s 273us/step - loss: 0.6741 - acc: 0.7943
Epoch 3/50
35/35 [==============================] - 0s 257us/step - loss: 0.6521 - acc: 0.8457
Epoch 4/50
35/35 [==============================] - 0s 149us/step - loss: 0.6294 - acc: 0.8229
Epoch 5/50
35/35 [==============================] - 0s 254us/step - loss: 0.6056 - acc: 0.8229
Epoch 6/50
35/35 [==============================] - 0s 144us/step - loss: 0.5803 - acc: 0.8286
Epoch 7/50
35/35 [==============================] - 0s 240us/step - loss: 0.5551 - acc: 0.8286
Epoch 8/50
35/35 [==============================] - 0s 154us/step - loss: 0.5298 - acc: 0.8229
Epoch 9/50
35/35 [==============================] - 0s 212us/step - loss: 0.5057 - acc: 0.8229
Epoch 10/50
35/35 [==============================] - 0s 149us/step - loss: 0.4823 - acc: 0.8229
Epoch 11/50
35/35 [==============================] 

#### Evaluate the model

In [103]:
loss, accuracy = model.evaluate(X_test, Y_test)
print('Accuracy: %.3f' % (accuracy*100))

15/15 [==============================] - 0s 129us/step
Accuracy: 78.667


### Let's have some insanity check

In [136]:
Y_predicted = model.predict(X_test)
pref_df = pd.DataFrame({
    'text' : [''.join([vocab_list[wordi]  if wordi > 0 else '.' for wordi in token]) for token in X_test],
    'fact' : [result.argmax() + 1 for result in Y_test],
    'pred' : [predict.argmax() + 1 for predict in Y_predicted]
}
)

In [137]:
pd.set_option('display.max_colwidth', 0) #default at 50
pref_df[pref_df.fact != pref_df.pred]

,text,fact,pred
1,............................................................................................................................................................................................................................................................................ร้าน.ร้านอาหารทะเลสด..เสาร์-อาทิตย์ราคาอาหารแพงร้านพนักงานเสิร์ฟอาหารรสชาติเยี่ยมผมสั่งสลัดกุ้งทอด..น้ำยำเปรี้ยวเปรี้ยวดีปลา.สดดีผมแนะนำร้านเหมาะงาน.,3,4
4,.....................................................................................................................................................................................................................................................ปอเปี๊ยะสดรู้สึกหากินร้านขายปอเปี๊ยะสดเจอขายภัตตาคารอร่อยร้าน.ราคา.30บาทซื้อกินรสชาติดี.กุนเชียงใส่.ราดน้ำจิ้มโอเคยอดเสิร์ฟ.เครื่องเคียงต้นหอม.เสิร์ฟโต๊ะนั่งกินร้านอารมณ์4ดาวป้าสำหรับอาหารเจอ.ป้าอาหาร.ทำปอเปี๊ยะใส่ถุงมือล้างมือผม.ดาว,3,4
5,..........................................................................................................................................................................................................เค้กเมืองไทยคนสงสัยคัพเค้กเค้กธรรมดาคัพเค้กรูปแบบเค้ก.เค้กจุดเด่นรสชาติเอกลักษณ์ผสมครีมชีสหลักทำหน้าเค้ก.คัพเค้กทาน.เค้กสุดท้ายตกแต่งแตกต่าง.คัพเค้กเค้กคัพเค้กตกแต่งสวยงาน.งานeventคู่บริษัท..สั่งคัพเค้กล่ะงานeventสำหรับร้าน.love.gatewayชั้นสามเดินทางสะดวกติด..ร้านรักดีร้านดู.ดูคึกคักดีผมทานรสรสชาติดีเค้กชุ่มชุ่มเค้กธรรมดาราคาดี59บาทชิ้นทาน.lemonadeร้านlovelemonadeรู้สึกเข้มข้นเปรี้ยวถึงใจหาทานประทับใจอ้อร้าน.ทำ...,5,4
9,........................................................................................................................................................................................................................................................................ร้านอาหาร..จอดรถ.รสชาติอาหารโอเคระดับนึงแปลกเป็ดอบกาแฟ.ราคาอาหารแพงสำหรับคุณภาพอาหารบรรยากาศ.กลางเริ่ด..ร้าน.ลองดูเลือกสำหรับคนชอบอาหารทะเลราคากลางรสชาติดีบรรยากาศโอเค,3,4
11,.......................................................................................................................................................................................................................................ร้านข้าวหมูกรอบผมฝากท้อง.ป้าย....ฟัง..นั่งกินจุดเด่นร้านราคาราคา.บาทนั่งกินร้านใส่ห่อร้านน้ำแข็งเปล่าหนังสือพิมพ์บริการฟรีแนะนำข้าวหมูกรอบหนังหมูกรอบอร่อยราดน้ำราดหวานกินพริกซีอิ๊วข้าวไก่ย่างราดน้ำ.เปรี้ยวหวานน้ำซุป1อร่อยผมอร่อย.ข้าวไก่.แดงร้านผมสอง.กรอบหมูกรอบ,2,4
14,"...........................................................................................................ก๋วยเตี๋ยวลูกชิ้นปลาย่าน.ร้านสิบปีกินประจำเวลา.บ่ายฮื่อก้วยอร่อยร้านอร่อยผม.ร้านดูเหมือน.ลอง:Dร้านห้าแยก.ตรงข้าม..ขับรถจอดรถริมถนนร้านคนกินพนักงานร้านเหมือน....กินตอนพักเที่ยงปัญหารองาน.เรื่อง:.ผมกินสั่ง1เส้น.แห้งยำใส่ฮื่อก้วย....รสชาติเยี่ยมเหมือนเส้นเหนียวดีลวกนิ่ม.ชอบกินเส้น..ผมชอบแน่2บะหมี่แห้งต้มยำสั่งเส้นโต๊ะรอบสั่งบะหมี่ลองอร่อยบะหมี่ร้านเส้นแบนร้านชามรสชาติ.แถมเผ็ดเครื่องปรุงเค้าใส่เตือน:กินเผ็ดกินต้มยำผมร้านเค้าอย่าใส่พริกเค้าปรุงรสอร่อย.สะใจชามผมสั่งเครื่องปกติเค้าใส่เกี๊ยว,ลูกชิ้นปลา,ฮื่อก้วยทอด,..,ลูกชิ้นปลาผมประทับใจฮื่อ.อิอิ.2ชามเงิน.บาทน้ำ.เทียบรสชาติเครื่องจ่ายเงิน.คนเดินสั่ง.ฮื่อก้วยทอดใส่ถุงน้ำจิ้มบ้านผมไหว.รอบหน้า:Dสรุปผมร้าน5ดาวสะอาด,ราคารสชาติอร่อยแพ้ร้าน",5,4
